# Calculate average PCRGlobWB supply using EE

* Purpose of script: This script will join the csv tables from GCS into one file using pandas
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170914

In [1]:
import time, datetime
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)

Y2017M10D10 UTC 16:05


In [2]:
GCS_VERSION = 17
OUTPUT_VERSION = 16

GCS_INPUT_PATH = "gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V%0.2d/" %(GCS_VERSION)
EC2_INPUT_PATH = "/volumes/data/Y2017M09D14_RH_merge_EE_results_V01/input"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M09D14_RH_merge_EE_results_V01/output"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"


STRING_TRIM = "V%0.2dee_export.csv" %(GCS_VERSION)
# e.g. IrrLinearWW_monthY2014M12V15ee_export.csv -> IrrLinearWW_monthY2014M12

#Aux files, do not change order i.e. zones, area, extra
AUXFILES = ["Hybas06",
            "area_30s_m2",
            "ones_30s"
           ]

DROP_COLUMNS = [".geo","system:index"]

OUTPUTFILENAME = "mergedZonalStatsEE_V%0.2d" %(OUTPUT_VERSION)

In [3]:
!rm -r {EC2_INPUT_PATH}
!rm -r {EC2_OUTPUT_PATH}

In [4]:
!mkdir -p {EC2_INPUT_PATH}
!mkdir -p {EC2_OUTPUT_PATH}

In [5]:
!gsutil cp -r {GCS_INPUT_PATH} {EC2_INPUT_PATH} 

Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/Hybas06V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/IrrLinearWN_monthY2014M01V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/IrrLinearWN_monthY2014M02V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/IrrLinearWN_monthY2014M03V17ee_export.csv...
- [4 files][  1.9 MiB/  1.9 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/IrrLinearWN_monthY2014M04V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/IrrLinearWN_monthY2014M05V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_

Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIndWW_monthY2014M11V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIndWW_monthY2014M12V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIndWW_yearY2014M12V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIrrWN_monthY2014M01V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIrrWN_monthY2014M02V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIrrWN_monthY2014M03V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIrrWN_monthY2014M04V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIrrWN_monthY2014M05V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIrrWN_monthY2014M06V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/PIrrWN_monthY2014M07V

Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/runoff_monthY2014M09V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/runoff_monthY2014M10V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/runoff_monthY2014M11V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/runoff_monthY2014M12V17ee_export.csv...
Copying gs://aqueduct30_v01/Y2017M09D11_RH_zonal_stats_EE_V17/runoff_yearY2014M12V17ee_export.csv...
| [159 files][ 88.1 MiB/ 88.1 MiB]    2.0 MiB/s                                 
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.


Operation completed over 159 objects/88.1 MiB.                                   


In [6]:
import pandas as pd
import os
import re

In [7]:
def createRegex(aList):
    return '|'.join(aList)

def prepareFile(oneFile):
        trimFileName = oneFile[:-len(STRING_TRIM)]
        d ={}
        d["df"] = pd.read_csv(os.path.join(folder,oneFile))
        d["df"] = prepareDf(d["df"])
        d["trimFileName"] = trimFileName
        return d         
        

def prepareDf(df):
    for column in df.columns:
        if re.search("PfafID",column):
            df2 = df.set_index(column)
            df2 = df2.drop(DROP_COLUMNS,1)        
            return df2
        


    
    


In [8]:
folder = os.path.join(EC2_INPUT_PATH,"Y2017M09D11_RH_zonal_stats_EE_V%0.2d/" %(GCS_VERSION)) 

In [9]:
files = os.listdir(folder)

## Process Auxiliary Datasets (PfafID, Area, Ones)

In [10]:
dAux ={}
for regex in AUXFILES:
    r = re.compile(regex)
    newList = filter(r.match, files)
    oneFile = list(newList)[0]
    dAux[regex] = prepareFile(oneFile)   

In [11]:
regex = createRegex(AUXFILES)

In [12]:
print(regex)

Hybas06|area_30s_m2|ones_30s


In [13]:
d ={}
dAux ={}
for oneFile in files: 
    trimFileName = oneFile[:-len(STRING_TRIM)]    
    if not re.search(regex,oneFile):
        d[trimFileName] = prepareFile(oneFile)
        
    elif re.search(regex,oneFile):
        dAux[trimFileName] = prepareFile(oneFile)
    
    else:
        print("Unrecognized file name, check STRING_TRIM variable")
        

In [14]:
dfLeft = dAux[AUXFILES[0]]["df"]

# Adding area to shapes

In [15]:
dAux[AUXFILES[1]]["df"]["total_%s" %(AUXFILES[1])] = dAux[AUXFILES[1]]["df"]["count_%s" %(AUXFILES[1])] * dAux[AUXFILES[1]]["df"]["mean_%s" %(AUXFILES[1])]

In [16]:
dfMerge = dAux[AUXFILES[0]]["df"].merge(dAux[AUXFILES[1]]["df"],
                       how="outer",
                       left_index=True,
                       right_index=True,
                       sort=True
                      )

In [17]:
for key, value in d.items():
    dfNew = value["df"].copy()
    # total new value = area in m^2 times mean flux 
    dfNew["total_volume_%s" %(value["trimFileName"])] = dAux[AUXFILES[1]]["df"]["total_%s" %(AUXFILES[1])] * value["df"]["mean_%s" %(value["trimFileName"])]
    
     
    
    dfMerge = dfMerge.merge(dfNew,
                           how="outer",
                           left_index=True,
                           right_index=True,
                           sort=True                   
                           )

In [18]:
dfMerge.head()

,count_Hybas06,mean_Hybas06,count_area_30s_m2,mean_area_30s_m2,total_area_30s_m2,count_IrrLinearWN_monthY2014M09,mean_IrrLinearWN_monthY2014M09,total_volume_IrrLinearWN_monthY2014M09,count_PIrrWW_monthY2014M01,mean_PIrrWW_monthY2014M01,...,total_volume_PIndWN_monthY2014M12,count_IrrLinearWW_monthY2014M07,mean_IrrLinearWW_monthY2014M07,total_volume_IrrLinearWW_monthY2014M07,count_PIrrWN_monthY2014M12,mean_PIrrWN_monthY2014M12,total_volume_PIrrWN_monthY2014M12,count_PLivWW_monthY2014M02,mean_PLivWW_monthY2014M02,total_volume_PLivWW_monthY2014M02
PfafID_Hybas06,,,,,,,,,,,,,,,,,,,,,
111011,2536,111011.0,2536,743658.186761,1.885917e+09,2536,0.000071,133567.156286,2536,0.000074,...,1.619210e+06,2536,0.000192,362371.436036,2536,0.000027,50631.847796,2536,1.161598e-07,219.067741
111012,3921,111012.0,3921,746186.462653,2.925797e+09,3921,0.000000,0.000000,3921,0.000000,...,2.675445e+05,3921,0.000000,0.000000,3921,0.000000,0.000000,3921,5.657729e-07,1655.336699
111013,1194,111013.0,1194,747422.836265,8.924229e+08,1194,0.000000,0.000000,1194,0.000000,...,2.476649e+05,1194,0.000000,0.000000,1194,0.000000,0.000000,1194,4.362127e-08,38.928622
111014,5605,111014.0,5605,750449.270450,4.206268e+09,5605,0.000000,0.000000,5605,0.000000,...,3.023537e+04,5605,0.000000,0.000000,5605,0.000000,0.000000,5605,1.812519e-07,762.394157
111015,21873,111015.0,21873,758792.279231,1.659706e+10,21769,0.000000,0.000000,21769,0.000000,...,3.593383e+06,21769,0.000000,0.000000,21769,0.000000,0.000000,21769,2.296991e-08,381.233072


In [25]:
dfMerge.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUTFILENAME+".csv"))

In [26]:
dfMerge.to_pickle(os.path.join(EC2_OUTPUT_PATH,OUTPUTFILENAME+".pkl"))

In [27]:
outputLocation = os.path.join(S3_OUTPUT_PATH,OUTPUTFILENAME)

In [28]:
!aws s3 cp --recursive {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH}

upload: ../../../../data/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V16.pkl to s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V16.pkl
upload: ../../../../data/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V16.csv to s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V16.csv


In [29]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:03:43.940126
